## Анализ

1. **Рекуррентные платежи** - это повторяющиеся по определенному рассписанию платежи, которые автоматически списываются с карты пользователя, после его согласия. Одним из ключевых и самых сложных этапов такого типа монетизации - "заставить" сделать самую первую покупку/подписку. Последующие этапы модели включают в себя много разнообразных подходов, которые помогут "удерживать" пользователя (улучшать продукт/улучшать условия подписки), но именно главный шаг - заинтерисовать клиента, чтобы он провёл первый платёж. Объяснить пользователю все возможности при подписке, допольнительные функции и тд. и тп. То есть, успех рекуррентных платежей может быть определен количеством этих самых "первых подписок". Конкретно с них и начинается цикл платежей, по каждому пользователю.
2. Одним из главных минусов является **доверие**. Не всегда пользователь готов предоставить все свои данные, для дальнейшего сотрудничества. Кроме того, имеют место быть проблемы со "случайной подпиской" (пользователь случайно подписался/забыл отписаться) или же не понял условия, на которых производится подписка.
3. Я думаю, что лучше всего было бы использовать **LTV (пожизненную ценность клиента)**. Это показатель прибыли, которую приносит пользователь за всё время работы с ним, нужно следить, чтобы расходы на привлечение были не больше доходов от всего взаимодействия с клиентом. Анализ по этой метрике поможет определить лояльных клиентов, оптимизировать схему по удержанию, понять поведенчиские факторы и также их улучшать .

## Моя модель 

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data_science_dataset_test_task.csv', delimiter = ',', index_col=0)
data.head()

,current_dt,user_id,scheduled_dt,root_order_id,gender,age,os_family,lt_user,hours_after_reg_buy,country_code,...,was_3ds,card_type,card_country_code,success_orders,not_success_orders,insufficients,root_gross,not_root_gross,old_model_probability,is_success
37,2019-05-12 05:40:15.355095,13801584,2019-06-09 00:00:00.000000,96601292,f,43,11,2,0,154,...,0,2.0,154,2,0,0,29.17,6.81,0.512364,1
45,2019-06-07 08:10:15.982701,16144816,2019-06-09 00:00:00.000000,116843195,f,25,11,4,76,3,...,0,2.0,3,2,1,1,0.47,0.46,0.133615,1
46,2019-06-07 08:10:15.982701,16467614,2019-06-09 00:00:00.000000,116847635,f,45,11,1,0,1,...,0,2.0,1,2,1,0,0.48,6.67,0.481923,0
47,2019-06-08 03:10:13.314904,16467614,2019-06-09 00:00:00.000000,116847635,f,45,11,2,0,1,...,0,2.0,1,2,1,0,0.48,6.67,0.485032,0
51,2019-06-07 08:10:15.982701,16482460,2019-06-09 00:00:00.000000,116953235,m,45,11,1,0,1,...,0,2.0,1,1,1,1,0.48,0.00,0.114275,0


In [3]:
corr = data.corr()
corr.style.background_gradient(cmap = 'coolwarm').set_precision(2)

,user_id,root_order_id,age,os_family,lt_user,hours_after_reg_buy,country_code,phone,cpa,retry_number,period,gross,currency_id,discount,psp,last_response_code,bank,was_3ds,card_type,card_country_code,success_orders,not_success_orders,insufficients,root_gross,not_root_gross,old_model_probability,is_success
user_id,1.00,1.00,0.02,0.01,0.03,-0.04,-0.01,-0.00,-0.02,-0.02,-0.04,0.01,-0.02,0.03,0.30,-0.03,-0.01,0.02,-0.00,-0.01,-0.07,-0.01,0.02,-0.11,-0.03,-0.05,0.01
root_order_id,1.00,1.00,0.02,0.01,0.03,-0.02,-0.01,-0.00,-0.02,-0.02,-0.03,0.02,-0.02,0.03,0.29,-0.03,-0.00,0.02,-0.00,-0.01,-0.07,-0.01,0.02,-0.12,-0.03,-0.05,0.01
age,0.02,0.02,1.00,0.05,0.03,0.13,0.05,0.07,0.10,-0.05,0.04,0.14,0.01,-0.04,0.05,0.06,0.08,0.00,0.08,0.04,0.08,-0.02,-0.05,0.07,0.08,0.16,0.06
os_family,0.01,0.01,0.05,1.00,0.03,0.02,-0.05,0.25,-0.06,0.02,-0.03,-0.06,0.01,0.03,-0.00,-0.04,-0.02,-0.03,0.00,-0.05,-0.00,0.03,0.05,-0.05,-0.02,-0.12,-0.04
lt_user,0.03,0.03,0.03,0.03,1.00,0.32,0.01,0.01,0.04,0.65,-0.61,-0.25,-0.00,0.61,0.08,-0.39,0.01,0.01,-0.01,0.01,0.05,0.37,0.39,0.07,-0.00,-0.33,-0.12
hours_after_reg_buy,-0.04,-0.02,0.13,0.02,0.32,1.00,0.04,0.02,0.01,-0.15,0.14,0.10,0.03,-0.14,-0.01,0.02,0.03,0.01,0.01,0.04,0.12,0.04,0.00,-0.00,0.02,0.01,0.00
country_code,-0.01,-0.01,0.05,-0.05,0.01,0.04,1.00,0.01,0.08,-0.02,0.07,0.72,0.34,-0.07,0.09,0.10,0.44,0.17,-0.01,0.92,0.07,0.04,-0.09,0.25,0.16,-0.00,0.00
phone,-0.00,-0.00,0.07,0.25,0.01,0.02,0.01,1.00,-0.01,-0.00,0.00,0.01,0.01,-0.00,-0.00,0.01,0.02,-0.00,0.00,0.01,0.00,-0.00,-0.00,-0.00,0.00,-0.01,-0.00
cpa,-0.02,-0.02,0.10,-0.06,0.04,0.01,0.08,-0.01,1.00,0.01,-0.01,0.12,0.03,0.01,0.07,0.03,0.05,-0.00,0.01,0.06,0.02,-0.01,-0.03,0.04,0.04,0.03,0.02
retry_number,-0.02,-0.02,-0.05,0.02,0.65,-0.15,-0.02,-0.00,0.01,1.00,-0.80,-0.35,-0.02,0.81,0.04,-0.47,-0.01,0.01,-0.02,-0.02,-0.13,0.44,0.49,-0.10,-0.06,-0.42,-0.15


In [4]:
categ = {0:'gender', 1:'age', 2:'period', 3:'discount', 4:'last_response_code', 5:'card_country_code'}
categ[0]

'gender'

In [5]:
for x in categ:
    gender_cat = data[categ[x]].value_counts(normalize=True)
    data[categ[x]+'_enc'] = data[categ[x]].map(gender_cat)
    data[[categ[x], categ[x]+'_enc']]

In [6]:
data.head()

,current_dt,user_id,scheduled_dt,root_order_id,gender,age,os_family,lt_user,hours_after_reg_buy,country_code,...,root_gross,not_root_gross,old_model_probability,is_success,gender_enc,age_enc,period_enc,discount_enc,last_response_code_enc,card_country_code_enc
37,2019-05-12 05:40:15.355095,13801584,2019-06-09 00:00:00.000000,96601292,f,43,11,2,0,154,...,29.17,6.81,0.512364,1,0.180706,0.026863,0.725067,0.760386,0.423515,0.021192
45,2019-06-07 08:10:15.982701,16144816,2019-06-09 00:00:00.000000,116843195,f,25,11,4,76,3,...,0.47,0.46,0.133615,1,0.180706,0.030921,0.725067,0.760386,0.474958,0.026057
46,2019-06-07 08:10:15.982701,16467614,2019-06-09 00:00:00.000000,116847635,f,45,11,1,0,1,...,0.48,6.67,0.481923,0,0.180706,0.027053,0.725067,0.760386,0.010184,0.693165
47,2019-06-08 03:10:13.314904,16467614,2019-06-09 00:00:00.000000,116847635,f,45,11,2,0,1,...,0.48,6.67,0.485032,0,0.180706,0.027053,0.725067,0.760386,0.010184,0.693165
51,2019-06-07 08:10:15.982701,16482460,2019-06-09 00:00:00.000000,116953235,m,45,11,1,0,1,...,0.48,0.00,0.114275,0,0.819294,0.027053,0.725067,0.760386,0.474958,0.693165


In [7]:
X_train, X_test = train_test_split(data, test_size=0.4, random_state=4)

In [8]:
X_train.head()

,current_dt,user_id,scheduled_dt,root_order_id,gender,age,os_family,lt_user,hours_after_reg_buy,country_code,...,root_gross,not_root_gross,old_model_probability,is_success,gender_enc,age_enc,period_enc,discount_enc,last_response_code_enc,card_country_code_enc
902847,2019-05-22 05:10:21.157491,14526455,2019-05-22 00:00:00.000000,102043932,m,45,11,4,6,1,...,0.48,0.00,0.055888,0,0.819294,0.027053,0.208050,0.208050,0.474958,0.693165
2376271,2019-06-06 05:10:19.898766,16342586,2019-06-08 00:00:00.000000,116205527,m,32,11,1,11,1,...,0.48,0.00,0.206829,0,0.819294,0.032859,0.725067,0.760386,0.010184,0.693165
1575253,2019-04-30 09:40:25.052529,12574178,2019-05-03 00:00:00.000000,88547338,m,31,11,1,18,1,...,0.48,0.00,0.159472,0,0.819294,0.031741,0.725067,0.760386,0.423515,0.693165
1642529,2019-05-10 07:40:19.774072,13448238,2019-06-09 00:00:00.000000,94101735,m,23,3,3,0,2,...,12.02,3.56,0.369331,0,0.819294,0.023162,0.725067,0.760386,0.423515,0.115622
2187587,2019-06-20 11:10:41.548276,17419749,2019-06-21 00:00:00.000000,126541392,f,35,11,4,54,2,...,0.54,0.00,0.180836,0,0.180706,0.039064,0.725067,0.760386,0.474958,0.115622


In [9]:
train_data = X_train[['gender_enc', 'age_enc','lt_user', 'hours_after_reg_buy', 'cpa', 'retry_number','period_enc', 
                   'gross','discount_enc', 'last_response_code_enc', 'card_country_code_enc', 'success_orders',
                  'not_success_orders', 'insufficients', 'root_gross', 'not_root_gross']]
train_data.head()

,gender_enc,age_enc,lt_user,hours_after_reg_buy,cpa,retry_number,period_enc,gross,discount_enc,last_response_code_enc,card_country_code_enc,success_orders,not_success_orders,insufficients,root_gross,not_root_gross
902847,0.819294,0.027053,4,6,0.000000,1,0.208050,9.706450,0.208050,0.474958,0.693165,1,5,3,0.48,0.00
2376271,0.819294,0.032859,1,11,1.198726,0,0.725067,19.971218,0.760386,0.010184,0.693165,1,1,0,0.48,0.00
1575253,0.819294,0.031741,1,18,1.077971,0,0.725067,19.782748,0.760386,0.423515,0.693165,1,0,0,0.48,0.00
1642529,0.819294,0.023162,3,0,0.000000,0,0.725067,11.568921,0.760386,0.423515,0.115622,3,0,0,12.02,3.56
2187587,0.180706,0.039064,4,54,3.000000,0,0.725067,8.608439,0.760386,0.474958,0.115622,1,1,1,0.54,0.00


In [10]:
train_labels = X_train.is_success
train_labels.head()

902847     0
2376271    0
1575253    0
1642529    0
2187587    0
Name: is_success, dtype: int64

In [11]:
test_data = X_test[['gender_enc', 'age_enc','lt_user', 'hours_after_reg_buy', 'cpa', 'retry_number','period_enc', 
                   'gross','discount_enc', 'last_response_code_enc', 'card_country_code_enc', 'success_orders',
                  'not_success_orders', 'insufficients', 'root_gross', 'not_root_gross']]
test_data.head()

,gender_enc,age_enc,lt_user,hours_after_reg_buy,cpa,retry_number,period_enc,gross,discount_enc,last_response_code_enc,card_country_code_enc,success_orders,not_success_orders,insufficients,root_gross,not_root_gross
1234092,0.819294,0.025195,2,0,0.000000,0,0.725067,18.514738,0.760386,0.423515,0.693165,1,0,0,0.39,0.0
2309032,0.819294,0.026849,3,2,0.000000,1,0.208050,8.612566,0.208050,0.474958,0.693165,1,2,2,0.38,0.0
357917,0.819294,0.027020,1,0,0.000000,0,0.725067,18.014664,0.760386,0.423515,0.693165,1,0,0,0.39,0.0
2210999,0.819294,0.025557,0,1,0.764509,0,0.725067,20.104082,0.760386,0.423515,0.693165,1,0,0,0.48,0.0
2219771,0.819294,0.036032,4,0,0.733315,2,0.031565,5.049500,0.031565,0.474958,0.693165,1,5,5,0.48,0.0


In [12]:
model = CatBoostClassifier(iterations=10) # классификатор
model.fit(train_data, train_labels) # обучение классификатора
prediction = model.predict(test_data) # передача тестовой выборки в модель
print(prediction) # вывод результата "предсказания"

Learning rate set to 0.5
0:	learn: 0.5347983	total: 246ms	remaining: 2.22s
1:	learn: 0.4840755	total: 325ms	remaining: 1.3s
2:	learn: 0.4631112	total: 406ms	remaining: 946ms
3:	learn: 0.4541798	total: 503ms	remaining: 754ms
4:	learn: 0.4505379	total: 584ms	remaining: 584ms
5:	learn: 0.4485392	total: 709ms	remaining: 473ms
6:	learn: 0.4475685	total: 849ms	remaining: 364ms
7:	learn: 0.4470109	total: 959ms	remaining: 240ms
8:	learn: 0.4465621	total: 1.05s	remaining: 117ms
9:	learn: 0.4461171	total: 1.16s	remaining: 0us
[0 0 0 ... 0 0 0]
